<a href="https://colab.research.google.com/github/DAOTHITHUNGA/CS114.K21.KHTN/blob/master/Sarcasm_Detec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###BÁO CÁO BÀI TẬP LỚN 
Đào Thị Thu Nga - 18521135


Mô tả bài toán : 
* data : Lấy headlines ở 2 trang chính là:
+ huffpost : chứa các tin không châm biếm 
+ TheOnion : chứa các tin châm biếm (is sarcastic)
* Gồm 3 thuộc tính: 
  + article_link : đường link dẫn tới bài báo 
  + is_sarcastic : xét có phải là châm biếm hay không với 2 option ( 1 nếu là châm biếm , 0 nếu là không châm biếm )
  + headlines : tên tiêu đề bài báo. 

### Từ tiêu đề của một bài báo, dự đoán bài báo đó có mang nội dung cà khịa hay không.

#### Cách thu thập dữ diệu:
-  Trước đây về Phát hiện Sarcasm chủ yếu sử dụng các bộ dữ liệu Twitter được thu thập bằng cách dựa trên hashtag nhưng các bộ dữ liệu đó rất nhiễu về LABEL và ngôn ngữ. Hơn nữa, nhiều tweet là trả lời các tweet khác và phát hiện sự mỉa mai trong những điều này đòi hỏi phải có sẵn các tweet theo ngữ cảnh. Để khắc phục những hạn chế liên quan đến  nhiễu trong bộ dữ liệu Twitter.

> Indented block


- Dữ liệu mới được thu thập để giảm độ nhiễu hơn so với dữ liệu thu thập từ Tweet từ trước. Dữ liệu được thu thập từ 2 trang tin tức chính:
    - Tập mang tính mỉa mai : Được thu thập từ trang tin tức TheOnion, từ 2 chủ đề chính là News in Brief và News in Photos
    - Tập không mang tính mỉa mai : Được thu thập từ trang tin tức HuffPost.

###Cách lấy dữ liệu từ 2 trang huffpost và TheOnion 
 
#Lấy file json của trang huffpost 
  - 1 lần  chỉ lấy được 500 line 
- Thực hiện các thao tác lấy file json của huffpost 
  - Lần 1 : Lấy file json của 500 headline ở https://www.huffpost.com/api/department/news/cards?page=1&limit=500
  - Lần 2: Lấy file json của 143 headline ở https://www.huffpost.com/api/department/news/cards?page=2&limit=500
  
#Lấy dữ liệu từ trang TheOnion 
 News in Brief và News in Photos

###CRAWL 2000 DỮ LIỆU 

### kết nối gg drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#import thư viện cần thiết 
import json
import csv
import pandas as pd

Xử lí file json của huffpost

In [4]:
#mở file chứa 158 headlines
with open('/content/huffpost.json') as f:
    part1 = json.load(f)
#mở file chứa 500 headlines
with open('/content/huffpost2.json') as f:
  part2 = json.load(f)
#tạo file data tên huffpost_headlines.
csv_file = open('huffpost_headlines.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['article_link','headlines', 'is_sarcastic'])
for article in part1["cards"]:
  headline = article["headlines"][0]["text"]
  article_link = article["headlines"][0]["url"]
  csv_writer.writerow([article_link,headline,0])
    
for article in part2["cards"]:
  headline = article["headlines"][0]["text"]
  article_link = article["headlines"][0]["url"]
  csv_writer.writerow([article_link,headline,0])
  #csv_file.close()

In [5]:
#đọc file huffpost_headlines.csv
huffpost_df = pd.read_csv("huffpost_headlines.csv")
huffpost_df.head()

,article_link,headlines,is_sarcastic
0,https://www.huffpost.com/entry/prince-william-...,Prince William Steps Out To Visit COVID-19 Vac...,0
1,https://www.huffpost.com/entry/supreme-court-d...,Supreme Court Bolsters Trump's Power Over Rapi...,0
2,https://www.huffpost.com/entry/george-floyd-di...,George Floyd Died A Month Ago. There Have Been...,0
3,https://www.huffpost.com/entry/carly-fiorina-2...,Former GOP Candidate Carly Fiorina Says She Wo...,0
4,https://www.huffpost.com/entry/colorado-elijah...,Colorado Governor Will Examine Death Of Elijah...,0


#Chuyển đổi từ file csv sang json

In [6]:
import csv
import json

csvfile = open('/content/huffpost_headlines.csv', 'r')
jsonfile = open('file.json', 'w')

fieldnames = ("article_link","headlines", "is_sarcastic")
reader = csv.DictReader( csvfile, fieldnames)
for row in reader:
    json.dump(row, jsonfile)
    jsonfile.write('\n')

#CRAWL dữ liệu từ TheOnion

In [7]:
import requests
from datetime import datetime
from bs4 import BeautifulSoup
import csv

brief_ids = [1591290240502,1590689280047,1589990700904,1589468640865,1588797540738,1588268460534,
             1587475140185,1586891580286,1586290320129,1585667400574,
             1584992760722,1584481080367,1583943300489,1583433840399,1583174280261,
             1582734480159,1582230180154,1581691680914,1581347940240,
              1580846940042,1580238780534,1579722060702,1579199040121,1578667500624,
              1576785600904,1576524120524,1575914280494,1575386760500,1574446260671,
              1573853280300,1573574280081,1573065060211,1572465120742,1572010200227,
              1571684700481,1571189760863,1570808100042,1570464000244,1570017600467,1569521460355,1569256200911
              ,1568653560896,1568141220732,1567616220385,1567090800228,1566577200032,
              1566249480367,1565716500732,1565282640227,1564762560287,1564500720500,1563984120219,1563541200063
             ,1563219960624,1562682300628]

photo_ids = [1590159240893,1588856760531,1587659280048,1586191620240,1584377100135,1582815480621,1582135980235,1580919900163,1579630380938
            ,1576675800080,1574713020966,1573241220619,1571856060048,1570206360590,1569267060072,1567600200783,1565965800980,
            1564618440716,1563282120008,1561045680206,1559133960526,1557427680731,1556198100070,
            1554825000123,1552924740401,1551208620373,1550168460985,1549035480816,
            1547746620251,1544817960808,1543529340295,1541717040221]

csv_file = open('TheOnion.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['headlines', 'is_sarcastic'])

24

In [8]:
def Onion_crawler(tag, ids, num_articles, label=1):
  count = 0
  for id in ids:
    source = requests.get("https://www.theonion.com/c/{}?startTime={}".format(tag,id)).text
    soup = BeautifulSoup(source, 'lxml')
    for article in soup.find_all('article'):
      headline = article.h2.text
      csv_writer.writerow([headline,1])
      count += 1
      if count == num_articles:
        print(f"Crawled {count} headlines from {tag} tag")
        return

In [9]:
Onion_crawler("news-in-brief", brief_ids, num_articles=750, label=1)
Onion_crawler("news-in-photos", photo_ids, num_articles=610, label=1)
csv_file.close()

Crawled 750 headlines from news-in-brief tag
Crawled 610 headlines from news-in-photos tag


In [10]:
#đọc file TheOnion
onion_df = pd.read_csv("TheOnion.csv")
onion_df.head(10)

,headlines,is_sarcastic
0,Drew Brees Admits He Doesn’t Trust Black Peopl...,1
1,Mark Zuckerberg Announces Virtual Roundtable W...,1
2,‘This Face Will Be The Last Thing You See Befo...,1
3,Primaried Steve King Glad He At Least Won’t Ha...,1
4,Police Defend Use Of Non-Lethal Rubber Tires O...,1
5,Democratic Leaders Announce That They’ve Learn...,1
6,"‘She Needs A Bible Now, Fuckwad—Yes, It’s For ...",1
7,Auntie Anne’s Breaks From Pack By Calling For ...,1
8,Trump Throws Garbage Can Through McDonald’s Wi...,1
9,Police Didn’t Spend Millions On Awesome Tank J...,1


In [11]:
#Chuyển đổi từ file csv sang file json
import csv
import json

csvfile = open('/content/TheOnion.csv', 'r')
jsonfile = open('Mydata_TheOnion.json', 'w')

fieldnames = ("headlines", "is_sarcastic")
reader = csv.DictReader( csvfile, fieldnames)
for row in reader:
    json.dump(row, jsonfile)
    jsonfile.write('\n')

Em trộn 2 file json của 2 trang Huffpost và TheOnion vào file 2000_test_headlines.json

### import thư viện cần thiết 

In [12]:
import pandas as pd, numpy as np, re, time
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

#đọc file json của kaggle (file train)

In [13]:
# Loading train data from json file
data_train_kaggle = pd.read_json("/content/drive/My Drive/hoctap/sarcasm/Sarcasm_Headlines_Dataset.json", lines = True)

#kiểm tra 

In [14]:
print(data_train_kaggle.isnull().any(axis = 0))
# Output :-
# ----------------------
# article_link    False
# headline        False
# is_sarcastic    False
# dtype: bool
# ----------------------

article_link    False
headline        False
is_sarcastic    False
dtype: bool


In [15]:
ps = PorterStemmer()
#Ham tien su li du lieu 
def pre_process(headlines):
# Relacing special symbols and digits in headline column
# re stands for Regular Expression
#LOẠI BỎ CÁC KÍ TỰ ĐẶC BIỆT KHÔNG CẦN THIẾT TRÊN DATA 
    headlines = headlines.apply(lambda s : re.sub('[^a-zA-Z]', ' ', s))
    # Tách câu thành từng từ, đưa các từ về từ gốc ban đầu (stemming data)
    #stemming is the process of reducing a word to its word stem that affixes to suffixes and prefixes or the roots of words known as a lemma.
    headlines = headlines.apply(lambda x: x.split())
    headlines = headlines.apply(lambda x : " ".join([ps.stem(word) for word in x]))
    return headlines

In [16]:
# getting features and labels
#Gán nhãn và trích features từ data đã có 
headline_kaggle = pre_process(data_train_kaggle['headline'])
labels_kaggle = data_train_kaggle['is_sarcastic']

In [17]:
#import TfidfVectorizer
#Một thuật toán rất phổ biến để chuyển đổi văn bản thành một đại diện có ý nghĩa của các con số được biểu diễn dưới dạng vector
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer()
tv.fit(headline_kaggle)
bag_of_word  = tv.vocabulary_
#in ra so tu moi chua trong thu vien 
print(bag_of_word)
print(len(bag_of_word))
def feature_extract(headlines):
    tv = TfidfVectorizer(vocabulary=bag_of_word, max_features=5000)
    headlines = list(headlines)
    features = tv.fit_transform(headlines).toarray()
    return features

{'former': 5584, 'versac': 16136, 'store': 14370, 'clerk': 2762, 'sue': 14540, 'over': 10677, 'secret': 13218, 'black': 1462, 'code': 2850, 'for': 5547, 'minor': 9536, 'shopper': 13501, 'the': 15032, 'roseann': 12764, 'reviv': 12527, 'catch': 2313, 'up': 15951, 'to': 15208, 'our': 10623, 'thorni': 15093, 'polit': 11410, 'mood': 9725, 'better': 1338, 'and': 470, 'wors': 16780, 'mom': 9672, 'start': 14256, 'fear': 5206, 'son': 13964, 'web': 16476, 'seri': 13313, 'closest': 2798, 'thing': 15073, 'she': 13426, 'will': 16644, 'have': 6633, 'grandchild': 6213, 'boehner': 1595, 'just': 7922, 'want': 16377, 'wife': 16626, 'listen': 8626, 'not': 10294, 'come': 2920, 'with': 16706, 'altern': 373, 'debt': 3694, 'reduct': 12243, 'idea': 7188, 'rowl': 12807, 'wish': 16701, 'snape': 13849, 'happi': 6548, 'birthday': 1434, 'in': 7294, 'most': 9771, 'magic': 8902, 'way': 16448, 'advanc': 153, 'world': 16771, 'women': 16735, 'fascin': 5168, 'case': 2281, 'eat': 4542, 'lab': 8253, 'grown': 6344, 'meat':

In [18]:
#Rút trích đặc trưng cho features của data_kaggle 
features_train_kaggle = feature_extract(headline_kaggle)

In [19]:
# Loading test data from THUNGADETHUONG WITH 2000 headlines
testdata = pd.read_json("/content/drive/My Drive/hoctap/sarcasm/2000_test_headlines.json", lines = True)

In [20]:
print(testdata.isnull().any(axis = 0))
#KIỂM TRA TẬP TEST 

article_link     True
headlines       False
is_sarcastic    False
dtype: bool


In [21]:
headline_testdata=pre_process(testdata['headlines'])
labels_testdata=testdata['is_sarcastic']

In [22]:
features_testdata= feature_extract(headline_testdata)

In [23]:
# model 1:-
# Using linear support vector classifier
lsvc = LinearSVC()
# training the model
lsvc.fit(features_train_kaggle, labels_kaggle)
# getting the score of train and test data
print(' linear support vector classifier')
print(lsvc.score(features_train_kaggle, labels_kaggle))
print(lsvc.score(features_testdata, labels_testdata))   

 linear support vector classifier
0.9552210865251414
0.7648809523809523


### Using Gaussuan Naive Bayes

In [24]:
# model 2:
# Using Gaussuan Naive Bayes
gnb = GaussianNB()
gnb.fit(features_train_kaggle, labels_kaggle)
print('Using Gaussuan Naive Bayes')
print(gnb.score(features_train_kaggle, labels_kaggle)) 
print(gnb.score(features_testdata, labels_testdata)) 

Using Gaussuan Naive Bayes
0.8059081208581377
0.6656746031746031


### Logistic Regression


In [25]:
# model 3:-
# Logistic Regression
lr = LogisticRegression()
lr.fit(features_train_kaggle, labels_kaggle)
print('Logistic Regression')
print(lr.score(features_train_kaggle, labels_kaggle))  
print(lr.score(features_testdata, labels_testdata))  

Logistic Regression
0.8957280317495975
0.7534722222222222


####Random Forest Classifier


In [26]:
# model 4:-
# Random Forest Classifier
rfc = RandomForestClassifier(n_estimators = 10, random_state = 0)
rfc.fit(features_train_kaggle, labels_kaggle)
print('Random Forest Classifier')
print(rfc.score(features_train_kaggle, labels_kaggle)) 
print(rfc.score(features_testdata, labels_testdata))   

Random Forest Classifier
0.989703845145831
0.7038690476190477


Do sự chênh lệnh quá lớn về dữ liệu trang mỉa mai (TheOnion) so với trang không mỉa mai (huffpost) nên việc sử dụng độ đo accuracy

### SỬ DỤNG ĐỘ ĐO PRECISION VÀ RECALL

In [27]:
Xtest = testdata['headlines']
ytest = testdata['is_sarcastic']
Xtest = feature_extract(Xtest)

In [28]:
from sklearn.metrics import classification_report
for cls in [lsvc,gnb, lr,rfc]:
    ypred = cls.predict(Xtest)
    yscore = cls.score(Xtest, ytest)
    print(classification_report(ytest, ypred))
    print(cls.__class__.__name__, yscore)

              precision    recall  f1-score   support

           0       0.52      0.80      0.63       656
           1       0.87      0.64      0.73      1360

    accuracy                           0.69      2016
   macro avg       0.69      0.72      0.68      2016
weighted avg       0.76      0.69      0.70      2016

LinearSVC 0.6904761904761905
              precision    recall  f1-score   support

           0       0.47      0.44      0.46       656
           1       0.74      0.76      0.75      1360

    accuracy                           0.66      2016
   macro avg       0.61      0.60      0.60      2016
weighted avg       0.65      0.66      0.66      2016

GaussianNB 0.6587301587301587
              precision    recall  f1-score   support

           0       0.54      0.81      0.65       656
           1       0.88      0.67      0.76      1360

    accuracy                           0.71      2016
   macro avg       0.71      0.74      0.70      2016
weighted avg   

### CHIA TẬP TRAIN VÀ TEST TRÊN TẬP DATA KAGGLE

In [29]:
features=features_train_kaggle
labels=labels_kaggle
#getting training and testing data
features_train_kaggle1, features_test_kaggle1, labels_train_kaggle1, labels_test_kaggle1 = train_test_split(features, labels, test_size = .05, random_state = 0)
del features
del features_train_kaggle

###Using linear support vector classifier

In [30]:
# model 1:-
# Using linear support vector classifier
lsvc = LinearSVC()
# training the model
lsvc.fit(features_train_kaggle1, labels_train_kaggle1)
# getting the score of train and test data
print(' Using linear support vector classifier')
print(lsvc.score(features_train_kaggle1, labels_train_kaggle1)) # 90.93
print(lsvc.score(features_test_kaggle1, labels_test_kaggle1))   # 83.75

 Using linear support vector classifier
0.9566468293067434
0.8323353293413174


####Logistic Regression

In [31]:
# model 2:-
# Logistic Regression
lr = LogisticRegression()
lr.fit(features_train_kaggle1, labels_train_kaggle1)
print('Logistic Regression')
print(lr.score(features_train_kaggle1, labels_train_kaggle1))   # 88.16
print(lr.score(features_test_kaggle1, labels_test_kaggle1))     # 83.08

Logistic Regression
0.8975288692704844
0.8353293413173652


### Random Forest Classifier


In [32]:
# model 3:-
# Random Forest Classifier
rfc = RandomForestClassifier(n_estimators = 10, random_state = 0)
rfc.fit(features_train_kaggle1, labels_train_kaggle1)
print('Random Forest Classifier')
print(rfc.score(features_train_kaggle1, labels_train_kaggle1))  # 98.82
print(rfc.score(features_test_kaggle1, labels_test_kaggle1))    # 79.71

Random Forest Classifier
0.9886099396996808
0.8016467065868264


# Using Gaussuan Naive Bayes


In [33]:
# model 4:
# Using Gaussuan Naive Bayes
#gnb = GaussianNB()
#gnb.fit(features_train_kaggle1, labels_train_kaggle1)
#print('Using Gaussuan Naive Bayes')
#print(gnb.score(features_train_kaggle1, labels_train_kaggle1)) 
#print(gnb.score(features_test_kaggle1, labels_test_kaggle1)) 

In [34]:
Xtest_kg = data_train_kaggle['headline']
ytest_kg = data_train_kaggle['is_sarcastic']
Xtest_kg = feature_extract(Xtest_kg)

In [35]:
from sklearn.metrics import classification_report
for cls in [lsvc, lr,rfc]:
    ypred = cls.predict(Xtest_kg)
    yscore = cls.score(Xtest_kg, ytest_kg)
    print(classification_report(ytest_kg, ypred))
    print(cls.__class__.__name__, yscore)

              precision    recall  f1-score   support

           0       0.86      0.90      0.88     14985
           1       0.86      0.82      0.84     11724

    accuracy                           0.86     26709
   macro avg       0.86      0.86      0.86     26709
weighted avg       0.86      0.86      0.86     26709

LinearSVC 0.8637912314201206
              precision    recall  f1-score   support

           0       0.85      0.86      0.85     14985
           1       0.82      0.80      0.81     11724

    accuracy                           0.84     26709
   macro avg       0.83      0.83      0.83     26709
weighted avg       0.84      0.84      0.84     26709

LogisticRegression 0.8355610468381445
              precision    recall  f1-score   support

           0       0.79      0.89      0.84     14985
           1       0.83      0.69      0.76     11724

    accuracy                           0.81     26709
   macro avg       0.81      0.79      0.80     26709
weighte

### Đối chiếu performance của model trên dataset đã cho và trên 2000 headine mới. Nêu nhận xét của nhóm về bài toán này.

- Từ kết quả trên thì performance của model trên dataset đã cho có kết quả cao hơn 2000 headline mới.
- Sử dụng bag of word không giúp hiểu nội dung headlines. Vì vậy cần tìm hiểu và test 1 số thuật toán khác như trong 1 bài báo sử dụng WORD2VEC MODEL và GloVe Embeddings để cải thiện performance hơn.
- Việc sử dụng word2vec model không chỉ xuất hiện tần số chữ như bag of word mà còn có xác suất số lần xuất hiện cùng nhau giữa các từ ngữ(có bao hàm ngữ cảnh).
- Việc sử dụng bag of word k đạt hiệu quả cao ở tập test là bộ dữ liệu hoàn toán mới là vì tập test có chứa những headline có từ mới nhiều chưa xuất hiện trong vocabulary của tập train.

###Mô tả cách dùng model đã train để viết một đoạn chương trình ngắn, thực hiện sacarsm detection cho một headline bất kỳ được nhập vào.

In [41]:
tmp = input()

tmp =pre_process(pd.Series(tmp))
tmp = feature_extract(tmp)
output = lr.predict(tmp)[0]
print("SARCASM" if output==1 else "NOT SARCASM")

THU NGA DE THUONG
SARCASM
